In [47]:
import pandas as pd
import sqlite3

## create a connection to the database using the library `sqlite3`

In [48]:
con = sqlite3.connect('../../datasets/checking-logs.sqlite')

## get the schema of the table `test`

In [49]:
pd.read_sql('PRAGMA table_info(test);', con)

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


## get only the first 10 rows of the tables `test` and `deadlines` to check what the table looks like


In [50]:
pd.read_sql('SELECT * FROM test LIMIT 10', con)

,uid,labname,first_commit_ts,first_view_ts
0,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


In [51]:
pd.read_sql('SELECT * FROM deadlines LIMIT 10', con, parse_dates='deadlines')

,index,labs,deadlines
0,0,laba04,2020-04-26 23:59:59
1,1,laba04s,2020-04-26 23:59:59
2,2,laba05,2020-05-03 23:59:59
3,4,laba06,2020-05-24 23:59:59
4,5,laba06s,2020-05-24 23:59:59
5,3,project1,2020-05-16 23:59:59


## find among all the users the `minimum` value of the delta between the first commit of the user and the deadline of the corresponding lab using only one query
  * do this by joining the table with the table `deadlines`
  * the difference should be displayed in `hours`
  * do not take the lab `project1` into account, it has longer deadlines and will be
  an outlier
  * the value should be stored in the dataframe `df_min` with the corresponding
  uid

In [52]:
df_min = pd.read_sql(
    '''
        SELECT uid, MIN(delta) AS delta
        FROM (
            SELECT 
                t.uid,
                CAST((julianday(t.first_commit_ts) - julianday(d.deadlines, 'unixepoch')) * 24
                     AS INTEGER
                ) AS delta
            FROM test AS t
            INNER JOIN deadlines AS d ON t.labname = d.labs
            WHERE NOT t.labname = 'project1'
        )
    ''',
    con,
)
df_min

,uid,delta
0,user_30,-202


## do the same thing, but for the `maximum`, using only one query, the dataframe name is `df_max`

In [53]:
df_max = pd.read_sql(
    '''
        SELECT uid, MAX(delta) AS delta
        FROM (
            SELECT 
                t.uid,
                CAST((julianday(t.first_commit_ts) - julianday(d.deadlines, 'unixepoch')) * 24
                     AS INTEGER
                ) AS delta
            FROM test AS t
            INNER JOIN deadlines AS d ON t.labname = d.labs
            WHERE NOT t.labname = 'project1'
        )
    ''',
    con,
)
df_max

,uid,delta
0,user_25,-2


## do the same thing but for the `average`, using only one query, this time your dataframe should not include the `uid` column, and the dataframe name is `df_avg`

In [54]:
df_avg = pd.read_sql(
    '''
        SELECT AVG(delta) AS average
        FROM (
            SELECT 
                t.uid,
                CAST((julianday(t.first_commit_ts) - julianday(d.deadlines, 'unixepoch')) * 24
                     AS INTEGER
                ) AS delta
            FROM test AS t
            INNER JOIN deadlines AS d ON t.labname = d.labs
            WHERE NOT t.labname = 'project1'
        )
    ''',
    con,
)
df_avg

,average
0,-89.125


## we want to test the hypothesis that the users who visited the newsfeed just a few
times have the lower delta between the first commit and the deadline. To do this,
you need to calculate the correlation coefficient between the number of pageviews
and the difference
  * using only one query, create a table with the columns: uid, avg_diff, pageviews
  * uid is the uids that exist in the test
  * avg_diff is the average delta between the first commit and the lab deadline per user
  * pageviews is the number of Newsfeed visits per user
  * do not take the lab ’project1’ into account
  * store it to the dataframe views_diff
  * use the Pandas method corr() to calculate the correlation coefficient between
the number of pageviews and the difference


In [55]:
views_df = pd.read_sql(
    '''
        SELECT 
            p.uid,
            AVG(t.delta) AS avg_diff,
            COUNT(p.uid) AS pageviews
        FROM (
            SELECT 
                t.uid,
                CAST((julianday(t.first_commit_ts) - julianday(d.deadlines, 'unixepoch')) * 24
                     AS INTEGER
                ) AS delta
            FROM test AS t
            INNER JOIN deadlines AS d ON t.labname = d.labs
            WHERE NOT t.labname = 'project1'
            GROUP BY t.uid 
        ) AS t
        JOIN pageviews AS p
        WHERE 
            p.uid LIKE 'user_%' AND
            p.uid = t.uid
        GROUP BY p.uid
            
    ''',
    con
)
views_df

,uid,avg_diff,pageviews
0,user_1,-6.0,28
1,user_10,-39.0,89
2,user_14,-200.0,143
3,user_17,-81.0,47
4,user_18,-4.0,3
5,user_19,-148.0,16
6,user_21,-126.0,10
7,user_25,-148.0,179
8,user_28,-98.0,149
9,user_3,-75.0,317


In [56]:
views_df[['avg_diff', 'pageviews']].corr()

,avg_diff,pageviews
avg_diff,1.000000,-0.062967
pageviews,-0.062967,1.000000


## Close the connection

In [57]:
con.close()